In [1]:
import json, requests, urllib, io
import time
import numpy as np
import pandas as pd
import boto3
import gzip
import sys
import snowflake.connector

import boto3
import base64
from botocore.exceptions import ClientError

from datetime import date
from pandas import json_normalize
from pandas.io.json import json_normalize

from src.aws import *
from src.utilityapi import * 
from src.snowflake import * 

In [7]:
# Query snowflake and write the bi.intervals table to a pandas dataframe
intervals = query_snowflake(
    """
    SELECT * FROM BI.INTERVALS;
    """
)
intervals.head()

,START_TS,END_TS,ZIP_CODE,METER_UID,UTILITY,UTILITY_SERVICE_ID,UTILITY_SERVICE_ADDRESS,UTILITY_METER_NUMBER,UTILITY_TARIFF_NAME,INTERVAL_START,INTERVAL_END,INTERVAL_KWH,NET_KWH,SOURCE,UPDATED,INTERVAL_TIMEZONE
0,2019-01-24 00:00:00,2019-01-24 00:15:00,90249,543449,SCE,3-001-3005-69,"15916 CRENSHAW BLVD UNIT 227, GARDENA, CA 90249",259000-027064,None,01/24/2019 00:00,01/24/2019 00:15,2.8860,2.8860,None,None,US/Pacific
1,2019-01-24 00:15:00,2019-01-24 00:30:00,90249,543449,SCE,3-001-3005-69,"15916 CRENSHAW BLVD UNIT 227, GARDENA, CA 90249",259000-027064,None,01/24/2019 00:15,01/24/2019 00:30,2.8860,2.8860,None,None,US/Pacific
2,2019-01-24 00:30:00,2019-01-24 00:45:00,90249,543449,SCE,3-001-3005-69,"15916 CRENSHAW BLVD UNIT 227, GARDENA, CA 90249",259000-027064,None,01/24/2019 00:30,01/24/2019 00:45,2.8860,2.8860,None,None,US/Pacific
3,2019-01-24 00:45:00,2019-01-24 01:00:00,90249,543449,SCE,3-001-3005-69,"15916 CRENSHAW BLVD UNIT 227, GARDENA, CA 90249",259000-027064,None,01/24/2019 00:45,01/24/2019 01:00,2.8860,2.8860,None,None,US/Pacific
4,2019-01-24 01:00:00,2019-01-24 01:15:00,90249,543449,SCE,3-001-3005-69,"15916 CRENSHAW BLVD UNIT 227, GARDENA, CA 90249",259000-027064,None,01/24/2019 01:00,01/24/2019 01:15,3.0120,3.0120,None,None,US/Pacific


In [29]:
# Example dataframe operations
## Select intervals for specific meter_uid and get basic statistics for INTERVAL_KWH
meter_uid = 543449 # enter meter_uid here

sample_df = intervals[intervals.METER_UID == meter_uid]
print("Mean INTERVAL_KWH:", sample_df.INTERVAL_KWH.mean())
print("Min INTERVAL_KWH:", sample_df.INTERVAL_KWH.min())
print("Max INTERVAL_KWH:", sample_df.INTERVAL_KWH.max())

# Grab the data where the max INTERVAL_KWH occurs
INTERVAL_KWH_max = sample_df[sample_df.INTERVAL_KWH == sample_df.INTERVAL_KWH.max()]
INTERVAL_KWH_max

Mean INTERVAL_KWH: 3.516797203198531
Min INTERVAL_KWH: 0.0000
Max INTERVAL_KWH: 15.6000


,START_TS,END_TS,ZIP_CODE,METER_UID,UTILITY,UTILITY_SERVICE_ID,UTILITY_SERVICE_ADDRESS,UTILITY_METER_NUMBER,UTILITY_TARIFF_NAME,INTERVAL_START,INTERVAL_END,INTERVAL_KWH,NET_KWH,SOURCE,UPDATED,INTERVAL_TIMEZONE
17915,2019-07-29 17:45:00,2019-07-29 18:00:00,90249,543449,SCE,3-001-3005-69,"15916 CRENSHAW BLVD UNIT 227, GARDENA, CA 90249",259000-027064,None,07/29/2019 17:45,07/29/2019 18:00,15.6000,15.6000,None,None,US/Pacific
17918,2019-07-29 18:30:00,2019-07-29 18:45:00,90249,543449,SCE,3-001-3005-69,"15916 CRENSHAW BLVD UNIT 227, GARDENA, CA 90249",259000-027064,None,07/29/2019 18:30,07/29/2019 18:45,15.6000,15.6000,None,None,US/Pacific


In [8]:
# Query snowflake and write the bi.bills table to a pandas dataframe
bills = query_snowflake(
    """
    SELECT * FROM BI.BILLS;
    """
)
bills.head()

,METER_UID,UTILITY,UTILITY_SERVICE_ID,UTILITY_BILLING_ACCOUNT,UTILITY_SERVICE_ADDRESS,UTILITY_METER_NUMBER,UTILITY_TARIFF_NAME,BILL_START_DATE,BILL_END_DATE,BILL_DAYS,BILL_STATEMENT_DATE,BILL_TOTAL_KWH,BILL_TOTAL,BILL_VOLUME,BILL_TOTAL_UNIT,DEMAND_KW
0,543449,SCE,3-001-3005-69,2-03-999-3324,15916 CRENSHAW BLVD UNIT 227 GARDENA CA 90249,259000-027064,TOU-GS-2-D-CPP,2020-06-18,2020-07-21,33,07/22/2020,9975.0000,2981.02,9975.0000,kwh,46.0000
1,543449,SCE,3-001-3005-69,2-03-999-3324,15916 CRENSHAW BLVD UNIT 227 GARDENA CA 90249,259000-027064,TOU-GS-2-D-CPP,2020-05-19,2020-06-18,30,06/19/2020,8373.0000,1845.50,8373.0000,kwh,30.0000
2,543449,SCE,3-001-3005-69,2-03-999-3324,15916 CRENSHAW BLVD UNIT 227 GARDENA CA 90249,259000-027064,TOU-GS-2-D-CPP,2020-04-17,2020-05-19,32,05/20/2020,8940.0000,1515.81,8940.0000,kwh,30.0000
3,543449,SCE,3-001-3005-69,2-03-999-3324,15916 CRENSHAW BLVD UNIT 227 GARDENA CA 90249,259000-027064,TOU-GS-2-D-CPP,2020-03-19,2020-04-17,29,04/18/2020,7613.0000,1211.76,7613.0000,kwh,27.0000
4,543449,SCE,3-001-3005-69,2-03-999-3324,15916 CRENSHAW BLVD UNIT 227 GARDENA CA 90249,259000-027064,TOU-GS-2-D-CPP,2020-02-19,2020-03-19,29,03/20/2020,8855.0000,1447.12,8855.0000,kwh,29.0000
